###### Use the below link only if using Google Colab

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#aida_means_link = drive.CreateFile({'id':'1Etu9qYdklVMBraeoerUWOl2hiNkGyxZj'})
#aida_means = aida_means_link.GetContentFile('aida_means.tsv')
aida_yoga_link = drive.CreateFile({'id':'1ThXRYBEGO-WbXX_cJiW_ojo36Wa5wCPp'})
aida_yoga = aida_yoga_link.GetContentFile('AIDA-YAGO2-dataset.tsv')
data_dict_link = drive.CreateFile({'id':'19TV-TnD_oF1Q57m-tF9iz2Ye7VRuFhFJ'})
data_dict = data_dict_link.GetContentFile('basic data dictionary.pkl')


In [ ]:
!pip install wikipedia

## Start from here

In [323]:
import wikipedia
import json
import csv
import pandas as pd
import numpy as np
import pickle
import time
import os

###### Exclude below aida means steps and start from file2

In [ ]:
data = pd.read_table('aida_means.tsv', delimiter="\t", header = None)
data.columns = ['col1', 'col2']
data.head()

In [ ]:
start = time.time()
#creating data dictionary for all the terms in the big file
data_dict = {}
for i in range(data.shape[0]):
   key = data.iloc[i,0]
   value = data.iloc[i,1]
   #key = "\"" + key + "\""
   value = json.loads("\"" + value + "\"")
   if key in data_dict:
       data_dict[key].append(value.replace("_"," "))
   else:
       data_dict[key] = [value]
print(time.time()-start)

In [ ]:
del data, aida_means, aida_means_link, aida_yoga, aida_yoga_link

In [ ]:
#len(data_dict)
'''
with open('basic data dictionary.pkl', 'wb') as f:
    pickle.dump(data_dict, f, pickle.HIGHEST_PROTOCOL)
'''
with open('input/basic data dictionary.pkl', 'rb') as f:
#with open('basic data dictionary.pkl', 'rb') as f:
    data_dict = pickle.load(f)

#len(trial_dict)
#data_dict[next(iter(trial_dict))]



In [ ]:
import gc
import os
gc.collect()

### Reading the mention 

In [338]:
file2 = 'input/AIDA-YAGO2-dataset.tsv'
#file2 = 'AIDA-YAGO2-dataset.tsv'
#writing the small file in list
data_list = list()
with open(file2,encoding="utf8") as csvfile:
    reader=csv.reader(csvfile, delimiter="\t")
    for row in reader:
        data_list.append(row)


In [333]:
data_list

[['-DOCSTART- (1 EU)'],
 ['EU', 'B', 'EU', '--NME--'],
 ['rejects'],
 ['German',
  'B',
  'German',
  'Germany',
  'http://en.wikipedia.org/wiki/Germany',
  '11867',
  '/m/0345h'],
 ['call'],
 ['to'],
 ['boycott'],
 ['British',
  'B',
  'British',
  'United_Kingdom',
  'http://en.wikipedia.org/wiki/United_Kingdom',
  '31717',
  '/m/07ssc'],
 ['lamb'],
 ['.'],
 [],
 ['Peter', 'B', 'Peter Blackburn', '--NME--'],
 ['Blackburn', 'I', 'Peter Blackburn', '--NME--'],
 [],
 ['BRUSSELS',
  'B',
  'BRUSSELS',
  'Brussels',
  'http://en.wikipedia.org/wiki/Brussels',
  '3708',
  '/m/0177z'],
 ['1996-08-22'],
 [],
 ['The'],
 ['European',
  'B',
  'European Commission',
  'European_Commission',
  'http://en.wikipedia.org/wiki/European_Commission',
  '9974',
  '/m/02q9k'],
 ['Commission',
  'I',
  'European Commission',
  'European_Commission',
  'http://en.wikipedia.org/wiki/European_Commission',
  '9974',
  '/m/02q9k'],
 ['said'],
 ['on'],
 ['Thursday'],
 ['it'],
 ['disagreed'],
 ['with'],
 ['Germa

In [340]:
data_list = [x for x in data_list if x]

In [341]:
#['None' if v is None else v for v in d]
data_df = pd.DataFrame(data_list)
data_df.columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
data_df.fillna(value=np.nan, inplace=True)
data_df.head(10)

,C1,C2,C3,C4,C5,C6,C7
0,-DOCSTART- (1 EU),NaN,NaN,NaN,NaN,NaN,NaN
1,EU,B,EU,--NME--,NaN,NaN,NaN
2,rejects,NaN,NaN,NaN,NaN,NaN,NaN
3,German,B,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
4,call,NaN,NaN,NaN,NaN,NaN,NaN
5,to,NaN,NaN,NaN,NaN,NaN,NaN
6,boycott,NaN,NaN,NaN,NaN,NaN,NaN
7,British,B,British,United_Kingdom,http://en.wikipedia.org/wiki/United_Kingdom,31717,/m/07ssc
8,lamb,NaN,NaN,NaN,NaN,NaN,NaN
9,.,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
data_df = data_df.dropna(subset=['C1'])

In [342]:
def get_clean_df(data_df):
    for i in range(len(data_df)):
        if '-DOCSTART-' in data_df.iloc[i,0]:
            for j in range(7):
                data_df.iloc[i,j] = '--split--here--'

    mention_content_corpur = ' '.join(list(data_df.C1))
    mention_content_list = mention_content_corpur.split('--split--here--')

    data_df = data_df.loc[data_df.C4.notnull()]
    data_df = data_df.loc[data_df.C4 != '--NME--']

    mention_list_corpur = '-*-'.join(list(data_df.C3))
    mention_list = mention_list_corpur.split('--split--here--')

    entity_list_corpur = '-*-'.join(list(data_df.C4))
    entity_list_corpur = json.loads('"' +  entity_list_corpur + '"')
    entity_list = entity_list_corpur.split('--split--here--')

    mention_list = [x for x in mention_list if x]
    entity_list = [x for x in entity_list if x]
    mention_content_list = [x for x in mention_content_list if x]

    for i in range(len(mention_list)):
        mention_list[i] = mention_list[i].split('-*-')
        mention_list[i] = [x for x in mention_list[i] if x]

    for i in range(len(entity_list)):
        entity_list[i] = entity_list[i].split('-*-')
        entity_list[i] = [x for x in entity_list[i] if x]
    
    data_df = data_df.loc[data_df.C4 != '--split--here--']
    
    return pd.DataFrame({'Mention_content': mention_content_list, 'Mention': mention_list, 'Entity': entity_list}), data_df

In [343]:
def split_data(data_list):
    #to find beginning and ending of training data

    for i in range(len(data_list)):
        for j in range(len(data_list[i])):
            if '1 EU' in data_list[i][j]:
                train_begin = i

    for i in range(len(data_list)):
        for j in range(len(data_list[i])):
            if '946 SOCCER' in data_list[i][j]:
                train_end = i

    train_data = data_list[train_begin:train_end]
    train_df = pd.DataFrame(train_data)
    train_df.columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
    train_df.fillna(value=np.nan, inplace=True)
    train_mention_df, train_df = get_clean_df(train_df)
    train_df = train_df.loc[train_df.C4 != '--NME--']
    
    #to find beginning and ending of testing A data
    for i in range(len(data_list)):
        for j in range(len(data_list[i])):
            if '947testa CRICKET' in data_list[i][j]:
                test_begin = i

    for i in range(len(data_list)):
        for j in range(len(data_list[i])):
            if '1162testa Dhaka' in data_list[i][j]:
                test_end = i

    testa_data = data_list[test_begin:test_end]
    testa_data = pd.DataFrame(testa_data)
    testa_data.columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
    testa_data.fillna(value=np.nan, inplace=True)
    testa_mention_df, testa_data = get_clean_df(testa_data)
    
    #to find beginning and ending of testing data
    for i in range(len(data_list)):
        for j in range(len(data_list[i])):
            if '1163testb SOCCER' in data_list[i][j]:
                test_begin = i

    for i in range(len(data_list)):
        for j in range(len(data_list[i])):
            if '1393testb SOCCER' in data_list[i][j]:
                test_end = i

    testb_data = data_list[test_begin:test_end]
    testb_data = data_list[test_begin:test_end]
    testb_data = pd.DataFrame(testb_data)
    testb_data.columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
    testb_data.fillna(value=np.nan, inplace=True)
    testb_mention_df, testb_data = get_clean_df(testb_data)
    
    return train_mention_df, testa_mention_df, testb_mention_df, train_df, testa_data, testb_data

In [344]:
train_mention_df, testa_mention_df, testb_mention_df, train_df, testa_df, testb_df = split_data(data_list) 

In [2]:
len(testa_mention_df.Doc.unique())

In [160]:
def extend_df(df, tag = 'train'):
    e_list = []
    m_list = []
    mc_list = []
    doc_list = []
    for i in range(len(df)):
        for j in range(len(df.iloc[i, 0])):
            doc_list.append(tag + '_doc_' + str(i))
            temp_entity = df.iloc[i, 0][j]
            temp_entity = temp_entity.replace('_', ' ').lower()
            e_list.append(temp_entity)
            m_list.append(df.iloc[i, 1][j])
            mc_list.append(df.iloc[i, 2])
    return pd.DataFrame({'Doc': doc_list, 'Entity': e_list, 'Mention': m_list, 'Mention_Content': mc_list})

In [193]:
train_mention_df = extend_df(train_mention_df, tag='train')
train_mention_df.head(10)

,Doc,Entity,Mention,Mention_Content
0,train_doc_0,germany,German,EU rejects German call to boycott British lam...
1,train_doc_0,united kingdom,British,EU rejects German call to boycott British lam...
2,train_doc_0,brussels,BRUSSELS,EU rejects German call to boycott British lam...
3,train_doc_0,european commission,European Commission,EU rejects German call to boycott British lam...
4,train_doc_0,european commission,European Commission,EU rejects German call to boycott British lam...
5,train_doc_0,germany,German,EU rejects German call to boycott British lam...
6,train_doc_0,united kingdom,British,EU rejects German call to boycott British lam...
7,train_doc_0,germany,Germany,EU rejects German call to boycott British lam...
8,train_doc_0,european union,European Union,EU rejects German call to boycott British lam...
9,train_doc_0,european union,European Union,EU rejects German call to boycott British lam...


In [194]:
len(train_mention_df.Doc.unique())

410

In [195]:
train_mention_df.drop_duplicates(inplace = True)
train_mention_df.reset_index(drop = True, inplace = True)

In [211]:
train_mention_df[train_mention_df.Doc == 'train_doc_120'].tail()

,Doc,Entity,Mention,Mention_Content
1690,train_doc_120,tokyo,TOKYO,Tokyo Soir - 1996 parent forecast . TOKYO 199...


In [212]:
train_mention_df_low = train_mention_df.iloc[0:1691]

In [213]:
len(list(train_mention_df_low.Mention.unique()))

1154

In [207]:
testa_mention_df = extend_df(testa_mention_df, tag='testa')
testa_mention_df.head(10)

IndexError: string index out of range

In [200]:
testa_mention_df.drop_duplicates(inplace = True)
testa_mention_df.reset_index(drop = True, inplace = True)

In [214]:
a = list(set(train_mention_df_low.Mention.unique()).intersection(set(testa_mention_df.Mention.unique())))

In [217]:
len(a)

246

In [228]:
len(testa_mention_df[testa_mention_df['Mention'].isin(a[0:20])].Doc.unique())

25

In [241]:
b = testa_mention_df[testa_mention_df['Mention'].isin(a[10:20])].Doc.unique()

In [242]:
#len(testa_mention_df[testa_mention_df['Mention'].isin(a[0:100])])
len(testa_mention_df[testa_mention_df['Doc'].isin(b)])

527

In [244]:
len(testa_mention_df)

1086

In [245]:
testa_mention_df_low = testa_mention_df[testa_mention_df['Doc'].isin(b)]

In [246]:
train_mention_df.to_csv('train_mention_df.csv', index = False)
train_mention_df_low.to_csv('train_mention_df_low.csv', index = False)
testa_mention_df.to_csv('testa_mention_df.csv', index = False)
testa_mention_df_low.to_csv('testa_mention_df_low.csv', index = False)

In [247]:
train_mention_df_low.head()

,Doc,Entity,Mention,Mention_Content
0,train_doc_0,germany,German,EU rejects German call to boycott British lam...
1,train_doc_0,united kingdom,British,EU rejects German call to boycott British lam...
2,train_doc_0,brussels,BRUSSELS,EU rejects German call to boycott British lam...
3,train_doc_0,european commission,European Commission,EU rejects German call to boycott British lam...
4,train_doc_0,germany,Germany,EU rejects German call to boycott British lam...


In [300]:
temp_mention_tr = train_mention_df_low.drop_duplicates(subset=['Doc', 'Mention_Content'])

In [301]:
temp_mention_tr.reset_index(drop = True, inplace = True)

In [302]:
temp_mention_tr.head()

,Doc,Entity,Mention,Mention_Content
0,train_doc_0,germany,German,EU rejects German call to boycott British lam...
1,train_doc_1,jimi hendrix,Hendrix,Rare Hendrix song draft sells for almost $ 17...
2,train_doc_2,people's republic of china,China,China says Taiwan spoils atmosphere for talks...
3,train_doc_3,people's republic of china,China,China says time right for Taiwan talks . BEIJ...
4,train_doc_4,germany,German,German July car registrations up 14.2 pct yr ...


In [311]:
temp_mention_ta = testa_mention_df_low.drop_duplicates(subset=['Doc', 'Mention_Content'])
temp_mention_ta.reset_index(drop = True, inplace = True)
temp_mention_ta.head()

,Doc,Entity,Mention,Mention_Content
0,testa_doc_0,leicestershire county cricket club,LEICESTERSHIRE,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFT...
1,testa_doc_8,us open (tennis),U.S. OPEN,TENNIS - FRIDAY 'S RESULTS FROM THE U.S. OPEN...
2,testa_doc_9,major league baseball,MAJOR LEAGUE,BASEBALL - MAJOR LEAGUE STANDINGS AFTER THURS...
3,testa_doc_10,major league baseball,MAJOR LEAGUE,BASEBALL - MAJOR LEAGUE RESULTS THURSDAY . NE...
4,testa_doc_13,milwaukee,MILWAUKEE,GOLF - LEADING SCORES AT GREATER MILWAUKEE OP...


In [304]:
import re
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

def tokenizer(text):
    text = text.replace('\\n',' ')
    text = text.replace('\'','')
    text = re.sub('[^A-Za-z]+', ' ', text)
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens

def get_tagged_data(df_):
    new_list = []
    for j in range(len(df_)):
        print(j)
        temp = tokenizer(df_.iloc[j,3])
        temp = nltk.pos_tag(temp)
        temp_list = []
        for i in range(len(temp)):
            if temp[i][1][:2] == 'NN':
                temp_list.append(temp[i][0])
        temp = list(set(temp_list))
        temp = ' '.join(temp)
        new_list.append(temp)
    df_['Mention_Content'] = pd.Series(new_list)
    return df_

In [305]:
tagged_train = get_tagged_data(temp_mention_tr)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


C:\Users\shubh\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [306]:
temp_mention_tr.iloc[0,3]

'lamb clearer March year animal measures veterinary sheepmeat issue Britain Commissioner National Palacio Commission conditions BRUSSELS spokesman Spanish cow month feed John concern Blackburn government Pas half Encephalopathy confidence France farmers report consumers Lloyd illness officials org humans http disease briefing health chains Werner sheep Zwingmann advice reports efforts Farm committee Peter chairman lead imports Loyola s mutton Welsh representative recommendation Europe ministers waste spleens risk action scientists move Fischler percent recommendations meeting beef news Union Farmers brains study van m EU consumer der radio contract Spongiform generalisation Wednesday countries Bonn proposal laboratory wasting call Franz Minister wikipedia h tonnes B Thursday BBC grounds alarm Bovine British danger committees food European Jones BSE brain farm Germany chief cords human NFU Nikolaus'

In [307]:
tagged_train.head()

,Doc,Entity,Mention,Mention_Content
0,train_doc_0,germany,German,lamb clearer March year animal measures veteri...
1,train_doc_1,jimi hendrix,Hendrix,Jimi Nottingham audience pearl sheet girlfrien...
2,train_doc_2,people's republic of china,China,Reuters hostility Shen Vice People comments Fo...
3,train_doc_3,people's republic of china,China,agency Association Straits executive Wednesday...
4,train_doc_4,germany,German,growth year figure increase General registrati...


In [312]:
tagged_testa = get_tagged_data(temp_mention_ta)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


C:\Users\shubh\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [314]:
tagged_testa

,Doc,Entity,Mention,Mention_Content
0,testa_doc_0,leicestershire county cricket club,LEICESTERSHIRE,title championship Moody Tim Paul requirements...
1,testa_doc_8,us open (tennis),U.S. OPEN,Sjeng Sukova Irina Pete Lisa Jan Veglio Tim si...
2,testa_doc_9,major league baseball,MAJOR LEAGUE,LEAGUE YORK FRANCISCO GB CINCINNATI Major CENT...
3,testa_doc_10,major league baseball,MAJOR LEAGUE,City LEAGUE YORK Results Major CALIFORNIA CAPS...
4,testa_doc_13,milwaukee,MILWAUKEE,Bryan Barr Stewart Loren Nolan Wisconsin Gorma...
5,testa_doc_17,berlin,BERLIN,Brahim Charles Kiptoo Tim Moen Daniel Cheruiyo...
6,testa_doc_28,canada,Canada,Reuters Ottawa August Bank billions Total liqu...
7,testa_doc_29,"st. louis, missouri",ST. LOUIS,Levoxyl hypothyroidism stock ST shares Chicago...
8,testa_doc_31,united states,U.S.,Lindsey year March dealers governor traders Ch...
9,testa_doc_33,chicago stock exchange,Chicago,UK coast tonne Ballantyne ex soymeal Chicago d...


In [317]:
train_final = train_mention_df_low[['Doc','Entity','Mention']].merge(tagged_train[['Doc', 'Mention_Content']], 
                                                                     how='left', on='Doc')

In [319]:
testa_final = testa_mention_df_low[['Doc','Entity','Mention']].merge(tagged_testa[['Doc', 'Mention_Content']], 
                                                                     how='left', on='Doc')

In [320]:
train_final.to_csv('train_final.csv', index = False)
testa_final.to_csv('testa_final.csv', index = False)

In [321]:
with open('train_mention.pkl', 'wb') as f:
    pickle.dump(train_final, f, pickle.HIGHEST_PROTOCOL)

with open('testa_mention.pkl', 'wb') as f:
    pickle.dump(testa_final, f, pickle.HIGHEST_PROTOCOL)

In [62]:
train_mentions_list = list(set(train_data.C3))
testa_mentions_list = list(set(testa_data.C3))
testb_mentions_list = list(set(testb_data.C3))
print(len(train_mentions_list))
print(len(testa_mentions_list))
print(len(testb_mentions_list))

2813
778
1130


In [64]:
train_data.to_csv('train_AIDA.csv', index = False)
testa_data.to_csv('testA_AIDA.csv', index = False)
testb_data.to_csv('testB_AIDA.csv', index = False)

### Reading priors data

In [67]:
priors_list = os.listdir('FADML-Project/priors/')

In [69]:
prior_dict = {}
for text in priors_list:
    prior_dict[text.split('-prior')[0]] = text

In [74]:
train_mention_context_link_dict = {}
for word in train_mentions_list:
  try:
    train_mention_context_link_dict[word] = prior_dict[word.lower()]
    #print(word , ' ==> ', prior_dict[word.lower()])
  except:
    print('Error in Word: ', word)
    pass

Error in Word:  Teresa
Error in Word:  House of Commons
Error in Word:  Association for Relations Across the Taiwan Straits


In [76]:
testa_mention_context_link_dict = {}
for word in testa_mentions_list:
  try:
    testa_mention_context_link_dict[word] = prior_dict[word.lower()]
    #print(word , ' ==> ', prior_dict[word.lower()])
  except:
    print('Error in Word: ', word)
    pass

In [77]:
testb_mention_context_link_dict = {}
for word in testb_mentions_list:
  try:
    testb_mention_context_link_dict[word] = prior_dict[word.lower()]
    #print(word , ' ==> ', prior_dict[word.lower()])
  except:
    print('Error in Word: ', word)
    pass

Error in Word:  UNHCR
Error in Word:  Desvonde Botes
Error in Word:  cocker spaniels


### Getting top 30 candidates of each mention

In [91]:
def get_top_30(filename):
    a = []
    b = []
    counter = 0
    with open('FADML-Project/priors/' + filename, encoding="utf8") as f:
        f = f.read()
        #print(f)
        f = f.split('\n')
        #print(counter)
        if len(f)%2 == 1:
            f = f[:-1]
        for line in f:
            #print(line)
            if counter%2 == 0:
                a.append(line)
            else:
                b.append(float(line))
            counter+=1

    trial = pd.DataFrame({'PageName': a, 'Probability': b})
    trial = trial.sort_values('Probability', ascending=False).head(30).reset_index(drop = True)
    return list(trial.PageName)

In [82]:
train_mention_context_link_dict

{'Tottenham': 'tottenham-prior.txt',
 'Fraser': 'fraser-prior.txt',
 'DOHUK': 'dohuk-prior.txt',
 'AMSTERDAM': 'amsterdam-prior (1).txt',
 'Aqaba': 'aqaba-prior.txt',
 'Arafat': 'arafat-prior.txt',
 'Juergen Klinsmann': 'juergen klinsmann-prior.txt',
 'Englishmen': 'englishmen-prior.txt',
 'Norman': 'norman-prior.txt',
 'Dean Headley': 'dean headley-prior.txt',
 'Africa': 'africa-prior.txt',
 'BRITISH MASTERS': 'british masters-prior.txt',
 'Peter Johansson': 'peter johansson-prior.txt',
 'BEIRUT': 'beirut-prior (1).txt',
 'Kankkunen': 'kankkunen-prior.txt',
 'Baltimore Orioles': 'baltimore orioles-prior.txt',
 'Australian': 'australian-prior.txt',
 'SOFIA': 'sofia-prior.txt',
 'Mike McCurry': 'mike mccurry-prior.txt',
 'Danish': 'danish-prior.txt',
 'Belmondo': 'belmondo-prior.txt',
 'UNITED NATIONS': 'united nations-prior.txt',
 'Bobby Bonilla': 'bobby bonilla-prior.txt',
 'Jonathan Lomas': 'jonathan lomas-prior.txt',
 'Netanyahu': 'netanyahu-prior.txt',
 'Annett Neumann': 'annett ne

In [94]:
top_30_pages_train_dict = {}
for word in list(train_mention_context_link_dict.keys()):
    try:
        top_30_pages_train_dict[word] = get_top_30(filename=train_mention_context_link_dict[word])
    except:
        print('Error in word: ', word)

Error in word:  Rangers
Error in word:  Civil War


In [97]:
top_30_pages_testa_dict = {}
for word in list(testa_mention_context_link_dict.keys()):
    try:
        top_30_pages_testa_dict[word] = get_top_30(filename=testa_mention_context_link_dict[word])
    except:
        print('Error in word: ', word)

In [98]:
top_30_pages_testb_dict = {}
for word in list(testb_mention_context_link_dict.keys()):
    try:
        top_30_pages_testb_dict[word] = get_top_30(filename=testb_mention_context_link_dict[word])
    except:
        print('Error in word: ', word)

Error in word:  Rangers
Error in word:  Finance
Error in word:  Cabinet


In [96]:
top_30_pages_train_dict['British']

['united kingdom',
 'great britain',
 'british people',
 'kingdom of great britain',
 'british empire',
 'england',
 'british army',
 'british raj',
 'royal navy',
 'british english',
 'india',
 'english people',
 'british armed forces',
 'british north america',
 'british language (celtic)',
 'british american',
 'british boxing board of control',
 'british antarctic territory',
 'british royal family',
 'roman britain',
 'british isles',
 'vogue (british magazine)',
 'british chilean',
 'british h class submarine',
 'kingdom of england',
 'east india company',
 'royal air force',
 'great britain national rugby league team',
 'british rule in burma',
 'thirteen colonies']

# Creating Wikipedia content dictionary

##### Start from here for wiki dictionary creation

In [2]:
'''
with open('input/top_30_pages_train.pkl', 'wb') as f:
    pickle.dump(top_30_pages_train_dict, f, pickle.HIGHEST_PROTOCOL)

with open('input/top_30_pages_train.txt', 'w') as f:
     f.write(json.dumps(top_30_pages_train_dict)) # use `json.loads` to do the reverse

with open('input/top_30_pages_testA.pkl', 'wb') as f:
    pickle.dump(top_30_pages_testa_dict, f, pickle.HIGHEST_PROTOCOL)

with open('input/top_30_pages_testA.txt', 'w') as f:
     f.write(json.dumps(top_30_pages_testa_dict)) # use `json.loads` to do the reverse

with open('input/top_30_pages_testB.pkl', 'wb') as f:
    pickle.dump(top_30_pages_testb_dict, f, pickle.HIGHEST_PROTOCOL)

with open('input/top_30_pages_testB.txt', 'w') as f:
     f.write(json.dumps(top_30_pages_testb_dict)) # use `json.loads` to do the reverse

'''

with open('input/top_30_pages_train.pkl', 'rb') as f:
    top_30_pages_train_dict = pickle.load(f)

with open('input/top_30_pages_testA.pkl', 'rb') as f:
    top_30_pages_testa_dict = pickle.load(f)

with open('input/top_30_pages_testB.pkl', 'rb') as f:
    top_30_pages_testb_dict = pickle.load(f)


In [3]:
with open('input/wikipedia_content_all.pkl', 'rb') as f:
    wikipedia_dict = pickle.load(f)

In [4]:
counter = 1
error_words = []
for word in list(top_30_pages_train_dict.keys()):
    print(counter, ' ===========================================================> ', word)
    counter+=1
    for i in range(len(top_30_pages_train_dict[word])):
        if top_30_pages_train_dict[word][i] not in wikipedia_dict:
            try:
                page = wikipedia.page(top_30_pages_train_dict[word][i])
                wikipedia_dict[top_30_pages_train_dict[word][i]] = page.content
            except:
                error_words.append(top_30_pages_train_dict[word][i])
                print('Error Word is: ', top_30_pages_train_dict[word][i])

1  ===========================================================>  Tottenham
2  ===========================================================>  Fraser


C:\Users\shubh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\shubh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error Word is:  richard fraser
Error Word is:  colin fraser
3  ===========================================================>  DOHUK
4  ===========================================================>  AMSTERDAM
5  ===========================================================>  Aqaba
6  ===========================================================>  Arafat
Error Word is:  hossam arafat
7  ===========================================================>  Juergen Klinsmann
8  ===========================================================>  Englishmen
9  ===========================================================>  Norman
10  ===========================================================>  Dean Headley
11  ===========================================================>  Africa
Error Word is:  africa (journal)
Error Word is:  africa (album)
12  ===========================================================>  BRITISH MASTERS
13  ===========================================================>  Peter Johansson
Error Word

Error Word is:  hms united (p44)
96  ===========================================================>  Marc Hottiger
97  ===========================================================>  WARSAW
98  ===========================================================>  Javier Frana
99  ===========================================================>  David Sesa
100  ===========================================================>  Faroe Islands
101  ===========================================================>  PARIS
Error Word is:  siege of paris
Error Word is:  paris (malcolm mclaren album)
102  ===========================================================>  Muster
103  ===========================================================>  British Open
104  ===========================================================>  Swerford
105  ===========================================================>  Tegel
Error Word is:  berlin-tegel railway station
106  ===========================================================>  Dickson
Erro

198  ===========================================================>  Ata-ur-Rehman
199  ===========================================================>  Iraqi Kurds
200  ===========================================================>  Galo Blanco
201  ===========================================================>  Socialist Party
Error Word is:  socialist party (sweden, 1929)
Error Word is:  spanish socialist workers' party
202  ===========================================================>  Rad
203  ===========================================================>  Diego Borrego
204  ===========================================================>  Brown
Error Word is:  scott brown (scottish footballer)
205  ===========================================================>  Aslan Maskhadov
206  ===========================================================>  SAN MATEO
207  ===========================================================>  Jiul Petrosani
208  ===========================================================>

...

Error Word is:  french empire
2391  ===========================================================>  Ellina Zvereva
2392  ===========================================================>  Charlton
Error Word is:  charlton
2393  ===========================================================>  Le Figaro
2394  ===========================================================>  Mutola
2395  ===========================================================>  neo-Nazi
2396  ===========================================================>  Chris Walker
Error Word is:  chris walker (baseball)
Error Word is:  chris walker (footballer)
2397  ===========================================================>  Ally McCoist
2398  ===========================================================>  Pesalai
2399  ===========================================================>  Split
Error Word is:  split dynasty
Error Word is:  split (song)
2400  ===========================================================>  Ukraine
2401  ====================

Error Word is:  preston
Error Word is:  preston, east lothian
2483  ===========================================================>  Oval
2484  ===========================================================>  Oldham
2485  ===========================================================>  Australians
2486  ===========================================================>  Chuck Finley
2487  ===========================================================>  Middle East
2488  ===========================================================>  Jordi Burillo
2489  ===========================================================>  Benjamin Netanyahu
2490  ===========================================================>  Detroit Tigers
2491  ===========================================================>  Jerusalem
2492  ===========================================================>  Arkansas State Police
2493  ===========================================================>  SBC Communications
2494  ====================================

Error Word is:  david martínez
Error Word is:  richard martinez
Error Word is:  james martinez
2580  ===========================================================>  KEANE
Error Word is:  jonathan keane
2581  ===========================================================>  Hanwha
2582  ===========================================================>  Younes El Aynaoui
2583  ===========================================================>  Wolverhampton
2584  ===========================================================>  Atletico Paranaense
2585  ===========================================================>  Calif
2586  ===========================================================>  Weston-super-Mare
Error Word is:  weston-super-mare railway station
2587  ===========================================================>  Jeff Tarango
2588  ===========================================================>  Paramaribo
2589  ===========================================================>  Jean-Philippe Fleurian
2590  ==

Error Word is:  brighton railway station, adelaide
2674  ===========================================================>  Torquay
2675  ===========================================================>  Cruyff
2676  ===========================================================>  NORTHAMPTON
Error Word is:  northampton, western australia
2677  ===========================================================>  Katarina Studenikova
2678  ===========================================================>  U.S. Open
2679  ===========================================================>  Albania
Error Word is:  albanian kingdom
Error Word is:  kingdom of albania
2680  ===========================================================>  Andre Markgraaff
2681  ===========================================================>  Gjon Buzuku
2682  ===========================================================>  Carlos Secretario
2683  ===========================================================>  Lisbet Stuer-Lauridsen
2684  ============

2770  ===========================================================>  Armstrong
2771  ===========================================================>  St Pauli
2772  ===========================================================>  Karina Habsudova
2773  ===========================================================>  SORRENTO
Error Word is:  sorrento, western australia
2774  ===========================================================>  Natasha Zvereva
2775  ===========================================================>  Taiwanese
2776  ===========================================================>  China Daily
2777  ===========================================================>  Sunday Bada
2778  ===========================================================>  Murat Yakin
2779  ===========================================================>  Stefka Kostadinova
2780  ===========================================================>  Groningen
2781  ===========================================================>  Robe

In [5]:
with open('input/wikipedia_content_new.pkl', 'wb') as f:
    pickle.dump(wikipedia_dict, f, pickle.HIGHEST_PROTOCOL)

In [6]:
counter = 1
error_words_testa = []
for word in list(top_30_pages_testa_dict.keys()):
    print(counter, ' ===========================================================> ', word)
    counter+=1
    for i in range(len(top_30_pages_testa_dict[word])):
        if top_30_pages_testa_dict[word][i] not in wikipedia_dict:
            try:
                page = wikipedia.page(top_30_pages_testa_dict[word][i])
                wikipedia_dict[top_30_pages_testa_dict[word][i]] = page.content
            except:
                error_words_testa.append(top_30_pages_testa_dict[word][i])
                print('Error Word is: ', top_30_pages_testa_dict[word][i])

1  ===========================================================>  Jansher Khan
2  ===========================================================>  AMERICAN LEAGUE EASTERN DIVISION
3  ===========================================================>  Bob Kennedy


C:\Users\shubh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\shubh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error Word is:  bob kennedy (american football)
4  ===========================================================>  Madagascar
Error Word is:  madagascar (video game)
5  ===========================================================>  Arafat
Error Word is:  hossam arafat
6  ===========================================================>  Collins
Error Word is:  sandy collins (tennis)
Error Word is:  bobby collins
7  ===========================================================>  Olabisi Afolabi
8  ===========================================================>  Russ Cochran
9  ===========================================================>  Switzerland
10  ===========================================================>  Paul Koech
11  ===========================================================>  Hussain
Error Word is:  altaf hussain
Error Word is:  sebeca zahra hussain
Error Word is:  mohammad hussain
Error Word is:  umair hussain
12  ===========================================================>  Cunningha

98  ===========================================================>  WARSAW
99  ===========================================================>  Ashes
Error Word is:  ashes (the prophecy album)
100  ===========================================================>  Magna
101  ===========================================================>  Milan
Error Word is:  milan (2004 film)
102  ===========================================================>  Jiri Novak
103  ===========================================================>  Transylvanian
104  ===========================================================>  RDR
105  ===========================================================>  Pro Bowl
106  ===========================================================>  Steve Stricker
107  ===========================================================>  German
108  ===========================================================>  Iraq
Error Word is:  majidi mall
109  ===========================================================>  Jer

189  ===========================================================>  William Tanui
190  ===========================================================>  EL SALVADOR
191  ===========================================================>  Mikael Tillstrom
192  ===========================================================>  Chandra Sturrup
193  ===========================================================>  MAJOR LEAGUE
194  ===========================================================>  Loren Roberts
195  ===========================================================>  Dan Jenson
196  ===========================================================>  Virgilijus Alekna
197  ===========================================================>  Steffi Graf
198  ===========================================================>  ATALANTA
199  ===========================================================>  Portugal
200  ===========================================================>  Paul Haarhuis
201  ================================

...

377  ===========================================================>  Andre Agassi
378  ===========================================================>  HENKE
379  ===========================================================>  Derbyshire
380  ===========================================================>  Emiliano Mondonico
381  ===========================================================>  National Bank of Poland
382  ===========================================================>  Kurdistan Democratic Party
Error Word is:  kurdistan democratic party (disambiguation)
383  ===========================================================>  Eurodollar
384  ===========================================================>  Ray Knight
Error Word is:  raymond knight (radio)
385  ===========================================================>  Jonathan Edwards
Error Word is:  jonathan edwards (athlete)
Error Word is:  jonathan edwards (archdeacon)
386  ===========================================================>  Ant

Error Word is:  major league baseball (video game)
466  ===========================================================>  Chesterfield
Error Word is:  chesterfield (disambiguation)
Error Word is:  chesterfield gorge
467  ===========================================================>  Fita Bayissa
468  ===========================================================>  Eurodollars
469  ===========================================================>  Jalal Talabani
470  ===========================================================>  Canadian
471  ===========================================================>  Bristol
Error Word is:  bristol, wisconsin
472  ===========================================================>  Rockies
Error Word is:  colligan/colligan/emmets gaa
473  ===========================================================>  Yekaterina Podkopayeva
474  ===========================================================>  Russian Federation
475  ===========================================================>

Error Word is:  newcastle (electoral district)
560  ===========================================================>  Ireland
Error Word is:  ireland national field hockey team
561  ===========================================================>  Eagles
Error Word is:  eagle (middle-earth)
562  ===========================================================>  Bahamas
563  ===========================================================>  Thomas Enqvist
564  ===========================================================>  Felix Mantilla
565  ===========================================================>  Nele Zilinskiene
566  ===========================================================>  Xiomara Rivero
567  ===========================================================>  Cyprus
568  ===========================================================>  Panama City
569  ===========================================================>  Dole
Error Word is:  charles e. dole
570  =================================================

661  ===========================================================>  Isel Lopez
662  ===========================================================>  CENTRAL DIVISION
Error Word is:  central division, western australia
663  ===========================================================>  MILWAUKEE
Error Word is:  milwaukee mustangs
Error Word is:  milwaukee brewers (minor league baseball team)
664  ===========================================================>  Kenny Harrison
665  ===========================================================>  Cincinnati Reds
666  ===========================================================>  Nigeria
667  ===========================================================>  Nyerere
668  ===========================================================>  Rod Beck
669  ===========================================================>  Tunbridge Wells
670  ===========================================================>  Cincinnati
671  ======================================================

756  ===========================================================>  Tegla Loroupe
757  ===========================================================>  Cardinals
Error Word is:  1941 st. louis cardinals season
758  ===========================================================>  Grand Slam
759  ===========================================================>  Swiss
760  ===========================================================>  Benes
761  ===========================================================>  Devon White
762  ===========================================================>  Phillies
763  ===========================================================>  Namibia
764  ===========================================================>  Ford
765  ===========================================================>  CLEVELAND
766  ===========================================================>  Alan Shearer
767  ===========================================================>  Czech Republic
768  ========================

In [7]:
with open('input/wikipedia_content_new.pkl', 'wb') as f:
    pickle.dump(wikipedia_dict, f, pickle.HIGHEST_PROTOCOL)

### Test B

In [8]:
counter = 1
error_words_testb = []
for word in list(top_30_pages_testb_dict.keys()):
    print(counter, ' ===========================================================> ', word)
    counter+=1
    for i in range(len(top_30_pages_testb_dict[word])):
        if top_30_pages_testb_dict[word][i] not in wikipedia_dict:
            try:
                page = wikipedia.page(top_30_pages_testb_dict[word][i])
                wikipedia_dict[top_30_pages_testb_dict[word][i]] = page.content
            except:
                error_words_testb.append(top_30_pages_testb_dict[word][i])
                print('Error Word is: ', top_30_pages_testb_dict[word][i])

1  ===========================================================>  Tottenham
2  ===========================================================>  Fraser


C:\Users\shubh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\shubh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error Word is:  richard fraser
Error Word is:  colin fraser
3  ===========================================================>  AMSTERDAM
4  ===========================================================>  AUSTRIA
5  ===========================================================>  Madagascar
Error Word is:  madagascar (video game)
6  ===========================================================>  Arafat
Error Word is:  hossam arafat
7  ===========================================================>  Dejan Koturovic
8  ===========================================================>  Korean
9  ===========================================================>  Luton
10  ===========================================================>  Kim Yoon-man
11  ===========================================================>  Jeremy Wotherspoon
12  ===========================================================>  European Commission
13  ===========================================================>  Mangosuthu Buthelezi
14  =========

Error Word is:  chantel kohl
102  ===========================================================>  Plymouth
103  ===========================================================>  Colin Cameron
Error Word is:  colin cameron (politician)
104  ===========================================================>  Pierre Van Hooydonk
105  ===========================================================>  Hanover
106  ===========================================================>  UEFA Cup
107  ===========================================================>  Bolton
Error Word is:  bolton (uk parliament constituency)
Error Word is:  bolton-on-dearne railway station
108  ===========================================================>  Serie A
109  ===========================================================>  Portsmouth
110  ===========================================================>  Pavel Bure
111  ===========================================================>  National League for Democracy
112  =========================

Error Word is:  appleton estate
201  ===========================================================>  Dutch
202  ===========================================================>  FIFA
203  ===========================================================>  Chicago Board of Trade
204  ===========================================================>  Scottish
Error Word is:  scottish
205  ===========================================================>  European
206  ===========================================================>  George Weah
207  ===========================================================>  Zimbabwe Open
208  ===========================================================>  Steve van Vuuren
209  ===========================================================>  Illinois
Error Word is:  2004–05 illinois fighting illini men's basketball team
210  ===========================================================>  BOMBAY
Error Word is:  bishop of bombay
211  =====================================================

...

Error Word is:  nicolai laudrup
707  ===========================================================>  Barnsley
708  ===========================================================>  Ebbw Vale
709  ===========================================================>  CHICAGO
710  ===========================================================>  Racing Santander
711  ===========================================================>  Rowson
712  ===========================================================>  Woolmer
713  ===========================================================>  Maccabi Herzliya
714  ===========================================================>  Albion
715  ===========================================================>  Sunderland
716  ===========================================================>  Michael DiVenuto
717  ===========================================================>  ANC
718  ===========================================================>  Holdsworth
Error Word is:  daniel holdsworth
719 

Error Word is:  florin constantinescu
806  ===========================================================>  Rama
Error Word is:  rama (video game)
807  ===========================================================>  Yugoslavia
Error Word is:  serbia national basketball team
808  ===========================================================>  PORT LOUIS
809  ===========================================================>  Mauritius
810  ===========================================================>  Suu Kyi
811  ===========================================================>  Italian
812  ===========================================================>  Oslo
813  ===========================================================>  International Confederation of Free Trade Unions
814  ===========================================================>  Hereford
815  ===========================================================>  Alloa
816  ===========================================================>  Ivan Zamorano
817  ==

Error Word is:  william hill (athlete)
Error Word is:  william hill (governor)
906  ===========================================================>  Arad
907  ===========================================================>  Goldberger
908  ===========================================================>  Carlos Belo
909  ===========================================================>  Peter Beardsley
910  ===========================================================>  Brechin
911  ===========================================================>  Fulham
912  ===========================================================>  Albania
Error Word is:  albanian kingdom
Error Word is:  kingdom of albania
913  ===========================================================>  Romania
914  ===========================================================>  National Hockey League
915  ===========================================================>  VOLENDAM
916  ===========================================================>  Sibiu
917

Error Word is:  toronto nationals
999  ===========================================================>  Harper
Error Word is:  bill harper
1000  ===========================================================>  Estonia
Error Word is:  estonia national basketball team
1001  ===========================================================>  Italy
1002  ===========================================================>  Fabrizio Ravanelli
1003  ===========================================================>  Bangui
1004  ===========================================================>  Chicago
1005  ===========================================================>  Clyde
1006  ===========================================================>  OPEC
1007  ===========================================================>  Glentoran
1008  ===========================================================>  Vatican
1009  ===========================================================>  Thailand
1010  ===========================================

Error Word is:  toronto nationals
1110  ===========================================================>  Celta Vigo
1111  ===========================================================>  Igor Kolyvanov
1112  ===========================================================>  Peter Nicol
1113  ===========================================================>  Bima Sakti
1114  ===========================================================>  JERUSALEM
1115  ===========================================================>  Pilkington Cup
1116  ===========================================================>  Axel Schulz
1117  ===========================================================>  Bournemouth
1118  ===========================================================>  Ahold
1119  ===========================================================>  Panhandle
1120  ===========================================================>  Kazuyoshi Funaki
1121  ===========================================================>  Compostela
1122  ==

In [9]:
with open('input/wikipedia_content_new.pkl', 'wb') as f:
    pickle.dump(wikipedia_dict, f, pickle.HIGHEST_PROTOCOL)

In [10]:
pd.DataFrame(error_words, columns=['PageName']).to_csv('input/Train_error_words.csv', index = False)
pd.DataFrame(error_words, columns=['PageName']).to_csv('input/Train_error_words.txt', index = False)

pd.DataFrame(error_words_testa, columns=['PageName']).to_csv('input/TestA_error_words.csv', index = False)
pd.DataFrame(error_words_testa, columns=['PageName']).to_csv('input/TestA_error_words.txt', index = False)

pd.DataFrame(error_words_testb, columns=['PageName']).to_csv('input/TestB_error_words.csv', index = False)
pd.DataFrame(error_words_testb, columns=['PageName']).to_csv('input/TestB_error_words.txt', index = False)

## POS Tagging of Wiki Dictionary

In [ ]:
with open(path_input + 'wikipedia_content_new.pkl', 'rb') as f:
    wikipedia_dict = pickle.load(f)

In [11]:
import re
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

def tokenizer(text):
    text = text.replace('\\n',' ')
    text = text.replace('\'','')
    text = re.sub('[^A-Za-z]+', ' ', text)
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens

In [12]:
def get_tagged_dict(dictionary):
  tic = time.time()
  temp_time = time.time()
  tagged_dict = {}
  count = 0
  n = len(dictionary)
  for word in dictionary.keys():
    count+=1
    if count%1000 == 0:
      print('Progress: ' + str(count) + ' of ' + str(n) + ' =====> Time Required: {0:.2f}'.format(time.time()-temp_time))
      temp_time = time.time()
    temp = tokenizer(dictionary[word])
    temp = nltk.pos_tag(temp)
    temp_list = []
    for i in range(len(temp)):
      if temp[i][1][:2] == 'NN':
        temp_list.append(temp[i][0])
    tagged_dict[word] = list(set(temp_list))
  print('Total Time Required: {0:.2f}'.format(time.time()-tic))
  return tagged_dict

In [13]:
tagged_wikidict = get_tagged_dict(wikipedia_dict)

Progress: 1000 of 30983 =====> Time Required: 78.04
Progress: 2000 of 30983 =====> Time Required: 76.38
Progress: 3000 of 30983 =====> Time Required: 78.14
Progress: 4000 of 30983 =====> Time Required: 67.75
Progress: 5000 of 30983 =====> Time Required: 69.05
Progress: 6000 of 30983 =====> Time Required: 78.25
Progress: 7000 of 30983 =====> Time Required: 73.06
Progress: 8000 of 30983 =====> Time Required: 75.04
Progress: 9000 of 30983 =====> Time Required: 73.05
Progress: 10000 of 30983 =====> Time Required: 77.12
Progress: 11000 of 30983 =====> Time Required: 77.23
Progress: 12000 of 30983 =====> Time Required: 74.58
Progress: 13000 of 30983 =====> Time Required: 94.71
Progress: 14000 of 30983 =====> Time Required: 83.49
Progress: 15000 of 30983 =====> Time Required: 67.35
Progress: 16000 of 30983 =====> Time Required: 71.97
Progress: 17000 of 30983 =====> Time Required: 69.67
Progress: 18000 of 30983 =====> Time Required: 67.57
Progress: 19000 of 30983 =====> Time Required: 64.12
Pr

In [14]:
with open('input/tagged_wikidict.pkl', 'wb') as f:
    pickle.dump(tagged_wikidict, f, pickle.HIGHEST_PROTOCOL)

In [15]:
len(tagged_wikidict)

30983

In [16]:
sum([len(v) for v in tagged_wikidict.values()])

9734646

# Appendix

### Get Cleaned df step by step output

In [47]:
data_df[data_df.C1.str.contains('DOCSTART')]['C2'] = data_df[data_df.C1.str.contains('DOCSTART')]['C1']

C:\Users\shubh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
data_df.iloc[0,0]

'-DOCSTART- (1 EU)'

In [53]:
for i in range(len(data_df)):
    if '-DOCSTART-' in data_df.iloc[i,0]:
        for j in range(7):
            data_df.iloc[i,j] = '--split--here--'

In [54]:
data_df

,C1,C2,C3,C4,C5,C6,C7
0,--split--here--,--split--here--,--split--here--,--split--here--,--split--here--,--split--here--,--split--here--
1,EU,B,EU,--NME--,NaN,NaN,NaN
2,rejects,NaN,NaN,NaN,NaN,NaN,NaN
3,German,B,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
4,call,NaN,NaN,NaN,NaN,NaN,NaN
5,to,NaN,NaN,NaN,NaN,NaN,NaN
6,boycott,NaN,NaN,NaN,NaN,NaN,NaN
7,British,B,British,United_Kingdom,http://en.wikipedia.org/wiki/United_Kingdom,31717,/m/07ssc
8,lamb,NaN,NaN,NaN,NaN,NaN,NaN
9,.,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
mention_content_corpur = ' '.join(list(data_df.C1))

In [60]:
mention_content_list = mention_content_corpur.split('--split--here--')

In [61]:
data_df = data_df.loc[data_df.C4.notnull()]
data_df = data_df.loc[data_df.C4 != '--NME--']
data_df

,C1,C2,C3,C4,C5,C6,C7
0,--split--here--,--split--here--,--split--here--,--split--here--,--split--here--,--split--here--,--split--here--
3,German,B,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
7,British,B,British,United_Kingdom,http://en.wikipedia.org/wiki/United_Kingdom,31717,/m/07ssc
14,BRUSSELS,B,BRUSSELS,Brussels,http://en.wikipedia.org/wiki/Brussels,3708,/m/0177z
18,European,B,European Commission,European_Commission,http://en.wikipedia.org/wiki/European_Commission,9974,/m/02q9k
19,Commission,I,European Commission,European_Commission,http://en.wikipedia.org/wiki/European_Commission,9974,/m/02q9k
26,German,B,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
32,British,B,British,United_Kingdom,http://en.wikipedia.org/wiki/United_Kingdom,31717,/m/07ssc
48,Germany,B,Germany,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
53,European,B,European Union,European_Union,http://en.wikipedia.org/wiki/European_Union,9317,/m/02jxk


In [80]:
mention_list_corpur = '-*-'.join(list(data_df.C3))
mention_list = mention_list_corpur.split('--split--here--')

entity_list_corpur = '-*-'.join(list(data_df.C4))
entity_list = entity_list_corpur.split('--split--here--')

In [88]:
mention_list = [x for x in mention_list if x]
entity_list = [x for x in entity_list if x]
mention_content_list = [x for x in mention_content_list if x]

In [127]:
#entity_list_corpur
entity_list_corpur = json.loads('"' +  entity_list_corpur + '"')

"--split--here---*-Germany-*-United_Kingdom-*-Brussels-*-European_Commission-*-European_Commission-*-Germany-*-United_Kingdom-*-Germany-*-European_Union-*-European_Union-*-United_Kingdom-*-European_Commission-*-European_Union-*-European_Union-*-Franz_Fischler-*-Franz_Fischler-*-United_Kingdom-*-France-*-Bovine_spongiform_encephalopathy-*-Spain-*-Loyola_de_Palacio-*-Loyola_de_Palacio-*-Loyola_de_Palacio-*-France-*-United_Kingdom-*-Bovine_spongiform_encephalopathy-*-United_Kingdom-*-Germany-*-United_Kingdom-*-Europe-*-Bonn-*-United_Kingdom-*-Germany-*-United_Kingdom-*-United_Kingdom-*---split--here---*-Jimi_Hendrix-*-London-*-United_States-*-Jimi_Hendrix-*-Jimi_Hendrix-*-Jimi_Hendrix-*-London-*-England-*-Nottingham-*-Jimi_Hendrix-*-Jimi_Hendrix-*-Australia-*---split--here---*-People's_Republic_of_China-*-Republic_of_China-*-Beijing-*-People's_Republic_of_China-*-Taipei-*-Taiwan_Strait-*-Taiwan_Strait-*-Ukraine-*-Republic_of_China-*-Lien_Chan-*-Lien_Chan-*-Beijing-*-People's_Republic_of_C

In [82]:
for i in range(len(mention_list)):
    mention_list[i] = mention_list[i].split('-*-')
    mention_list[i] = [x for x in mention_list[i] if x]

In [83]:
for i in range(len(entity_list)):
    entity_list[i] = entity_list[i].split('-*-')
    entity_list[i] = [x for x in entity_list[i] if x]

#### Earlier Data extraction run

In [5]:
wikipedia_dict = {}
counter = 1
error_words = []
for word in list(top_30_pages_train_dict.keys()):
    print(counter, ' ==> ', word)
    counter+=1
    for i in range(len(top_30_pages_train_dict[word])):
        if top_30_pages_train_dict[word][i] not in wikipedia_dict:
            try:
                page = wikipedia.page(top_30_pages_train_dict[word][i])
                wikipedia_dict[top_30_pages_train_dict[word][i]] = page.content
            except:
                error_words.append(top_30_pages_train_dict[word][i])
                print('Error Word is: ', top_30_pages_train_dict[word][i])

1  ==>  Ahmed Yassin
2  ==>  Ransart
3  ==>  Brazilian
4  ==>  Interfax
5  ==>  Jos Verstappen
6  ==>  Michigan
7  ==>  Jonathan Lomas
8  ==>  Toronto Blue Jays
9  ==>  Montpellier
10  ==>  Diario 16
11  ==>  Bruno Thiry
12  ==>  Shaun Pollock
13  ==>  Yingkou
14  ==>  AJACCIO
15  ==>  The Times
Error Word is:  the times (south africa)
16  ==>  HELSINKI
17  ==>  African
18  ==>  Andre Snyman
19  ==>  Paulo Sousa
20  ==>  Heinrich


C:\Users\shubh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\shubh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error Word is:  heinrich debus
21  ==>  Hosni Mubarak
22  ==>  Helsinki
23  ==>  Jose Rivero
24  ==>  BUENOS AIRES
25  ==>  Husaberg
26  ==>  Ian Woosnam
27  ==>  Wisconsin
28  ==>  Twins
29  ==>  Widzew Lodz
30  ==>  Toshiba Classic
31  ==>  John Cook
Error Word is:  john cook (musician)
32  ==>  D.A. Weibring
33  ==>  Cedric Pioline
34  ==>  Nepal
35  ==>  CHANDA RUBIN
36  ==>  St Louis
37  ==>  Swe
38  ==>  Stoke
Error Word is:  stoke-on-trent railway station
Error Word is:  stoke railway station
39  ==>  Kevin Seitzer
40  ==>  Chang
Error Word is:  chang (film)
Error Word is:  chang ucchin
Error Word is:  peter i. chang
41  ==>  Denmark
42  ==>  Regina Jacobs
43  ==>  Finland
44  ==>  Durham
Error Word is:  durham
Error Word is:  durham (electoral district)
Error Word is:  durham (district)
Error Word is:  electoral district of durham
45  ==>  Hapoel Ironi
46  ==>  University Hospital
47  ==>  Thomas Enqvist
48  ==>  Uppsala
49  ==>  Neil Hodgson
50  ==>  PAKISTAN
51  ==>  Glasgow


306  ==>  Foreign Ministry
Error Word is:  foreign ministry of austria-hungary
307  ==>  ATHENS
308  ==>  RUGBY UNION
309  ==>  Joost van der Westhuizen
310  ==>  Davis Cup
Error Word is:  chile davis cup team
311  ==>  Marcio Santos
312  ==>  Phnom Penh
313  ==>  Columbia University
314  ==>  Suleimaniya
315  ==>  First Pacific
316  ==>  Juergen Klinsmann
317  ==>  Ian Salisbury
318  ==>  AUSTRIA
319  ==>  Mark Rutherford
Error Word is:  mark rutherford (footballer)
320  ==>  Lidl
321  ==>  TOGO
322  ==>  Derek Jeter
323  ==>  Colombo
324  ==>  Bob May
325  ==>  Reserve Bank of India
326  ==>  St Kilda
327  ==>  Sudan
328  ==>  Kenneth Eriksson
329  ==>  BALTIMORE
330  ==>  Africa
Error Word is:  africa (album)
Error Word is:  africa (journal)
331  ==>  AEK Larnaca
332  ==>  Iraqi Kurds
333  ==>  Fernando Meligeni
334  ==>  Exeter
335  ==>  Damascus
Error Word is:  battle of damascus
336  ==>  Royals
337  ==>  Jorge Cadete
338  ==>  Colorado Rockies
339  ==>  Etruria
340  ==>  Miss Un

575  ==>  Natasha Zvereva
576  ==>  Jason Weaver
577  ==>  Laban Rotich
578  ==>  Ed Vosberg
579  ==>  Inez Turner
580  ==>  Jose Valentin
581  ==>  Joel Stransky
582  ==>  Borussia Moenchengladbach
583  ==>  Social Democrats
584  ==>  Alan Shearer
585  ==>  Sampras
586  ==>  Toronto-based
587  ==>  William Tanui
588  ==>  KANSAS CITY
589  ==>  Ramle
590  ==>  Fox Broadcasting
591  ==>  Philippoussis
592  ==>  West Bank
Error Word is:  palestine
593  ==>  Socialist Party
Error Word is:  socialist party (sweden, 1929)
Error Word is:  spanish socialist workers' party
594  ==>  Akron
Error Word is:  akron, indiana
595  ==>  Tami Whitlinger Jones
596  ==>  Bordeaux
597  ==>  Dynamo Moscow
Error Word is:  dynamo moscow (women's basketball)
598  ==>  ANGOLA
599  ==>  Danish
600  ==>  Taiwan
601  ==>  Ralf Kelleners
602  ==>  Monica Seles
603  ==>  Feyenoord Rotterdam
604  ==>  Seles
605  ==>  Palmeiras
606  ==>  Wolverhampton
607  ==>  Otelul Galati
608  ==>  Arjuna Ranatunga
609  ==>  Yevge

...

Error Word is:  jody morris
Error Word is:  morris county, kansas
Error Word is:  benny morris
Error Word is:  morris (electoral district)
Error Word is:  morris, oklahoma
Error Word is:  morris cohen (soviet spy)
Error Word is:  uss morris (dd-417)
Error Word is:  morris, alabama
Error Word is:  lee morris (footballer)
Error Word is:  william morris
Error Word is:  archie morris
Error Word is:  nasief morris
Error Word is:  morris, georgia
Error Word is:  morris & company
Error Word is:  hugh morris (footballer)
Error Word is:  edmund morris (writer)
Error Word is:  morris (village), new york
Error Word is:  morris tribunal
Error Word is:  morris township, grundy county, illinois
Error Word is:  brett morris
2506  ==>  Derbyshire
Error Word is:  derbyshire
Error Word is:  derbyshire county cricket club
Error Word is:  derbyshire (uk parliament constituency)
Error Word is:  matt derbyshire
Error Word is:  mv derbyshire
Error Word is:  derbyshire county football association
Error Word i

Error Word is:  coat of arms of saudi arabia
Error Word is:  foreign relations of saudi arabia
Error Word is:  saudi arabia at the 2006 fei world equestrian games
Error Word is:  prime minister of saudi arabia
Error Word is:  capital punishment in saudi arabia
Error Word is:  telephone numbers in saudi arabia
Error Word is:  saudi list of most wanted suspected terrorists
Error Word is:  saudi arabian records in athletics
Error Word is:  saudi arabia national handball team
Error Word is:  elections in saudi arabia
Error Word is:  human rights in saudi arabia
Error Word is:  saudi arabia men's national ice hockey team
Error Word is:  saudi arabia at the asian games
Error Word is:  ministry of interior (saudi arabia)
Error Word is:  line of succession to the saudi arabian throne
Error Word is:  second saudi state
Error Word is:  first saudi state
2542  ==>  Jakarta
Error Word is:  soekarno–hatta international airport
2543  ==>  Coast guard
2544  ==>  Peter Martin
Error Word is:  peter mar

Error Word is:  tanzania at the 1972 summer olympics
2574  ==>  Byron Black
Error Word is:  byron black
2575  ==>  GERMAN OPEN
2576  ==>  Kaolack
Error Word is:  kaolack
Error Word is:  kaolack department
Error Word is:  kaolack region
2577  ==>  Greer
Error Word is:  greer, south carolina
Error Word is:  ross greer
Error Word is:  mary k. greer
2578  ==>  David Sesa
Error Word is:  david sesa
2579  ==>  National League West
2580  ==>  Mediterranean
Error Word is:  mediterranean sea
Error Word is:  mediterranean basin
Error Word is:  mediterranean region, turkey
Error Word is:  mediterranean race
Error Word is:  battle of the mediterranean
Error Word is:  mediterranean fleet
Error Word is:  mediterranean diet
Error Word is:  mediterranean squadron (united states)
Error Word is:  mediterranean sea (oceanography)
Error Word is:  mediterranean naval engagements during world war i
Error Word is:  mediterranean shipping company s.a.
Error Word is:  mediterranean revival architecture
Error W

Error Word is:  jimmy joe roche
Error Word is:  patrick roche
2612  ==>  Hendrix
Error Word is:  hendrix college
Error Word is:  hendrix, oklahoma
Error Word is:  hendrix (crater)
Error Word is:  howard v. hendrix
Error Word is:  terri hendrix
Error Word is:  joseph c. hendrix
Error Word is:  friedrich hendrix
Error Word is:  wanda hendrix
Error Word is:  james r. hendrix
Error Word is:  leon hendrix
Error Word is:  eugene russell hendrix
Error Word is:  david hendrix
Error Word is:  veronica hendrix
Error Word is:  leslie hendrix
Error Word is:  gary hendrix
Error Word is:  jesse hendrix
Error Word is:  john w. hendrix
Error Word is:  richard hendrix
Error Word is:  neal hendrix
Error Word is:  don hendrix
Error Word is:  brunhilde hendrix
Error Word is:  paul f. hendrix
Error Word is:  arthur hendrix
Error Word is:  rick hendrix
Error Word is:  jenny hendrix
Error Word is:  freman hendrix
Error Word is:  manny hendrix
Error Word is:  elaine hendrix
Error Word is:  claude hendrix
2613

Error Word is:  moreau river
2706  ==>  Said
2707  ==>  Boskalis
2708  ==>  Milwaukee
Error Word is:  milwaukee mustangs
Error Word is:  milwaukee brewers (minor league baseball team)
2709  ==>  Bohinen
2710  ==>  BEIRUT
2711  ==>  Meluawati
2712  ==>  Tubmanburg
2713  ==>  Thomas Bscher
2714  ==>  Tibetan
2715  ==>  Boston
Error Word is:  boston red stockings
2716  ==>  United Kingdom
2717  ==>  South African
Error Word is:  international cricketers of south african origin
Error Word is:  the star (south africa)
2718  ==>  Almere
2719  ==>  Whitehurst
Error Word is:  agent m
2720  ==>  American
2721  ==>  Nicaraguan
2722  ==>  Bayer Leverkusen
2723  ==>  Bscher
2724  ==>  Frankie Fredericks
2725  ==>  Forfar
Error Word is:  angus
2726  ==>  Peru
2727  ==>  Clarence Rose
2728  ==>  Don Wengert
2729  ==>  George Joulwan
2730  ==>  Ken Hill
2731  ==>  Serbia
2732  ==>  NEW DELHI
2733  ==>  Angolan
2734  ==>  Rubens Barrichello
2735  ==>  Carl Fogarty
2736  ==>  Poles
2737  ==>  1860 Muni

In [6]:
with open('input/wikipedia_content.pkl', 'wb') as f:
    pickle.dump(wikipedia_dict, f, pickle.HIGHEST_PROTOCOL)

with open('input/wikipedia_content.txt', 'w') as f:
     f.write(json.dumps(wikipedia_dict)) # use `json.loads` to do the reverse

In [7]:
import sys
sys.getsizeof(wikipedia_dict)

589928

In [8]:
counter = 1
error_words_test = []
for word in list(top_30_pages_test_dict.keys()):
    print(counter, ' ==> ', word)
    counter+=1
    for i in range(len(top_30_pages_test_dict[word])):
        if top_30_pages_test_dict[word][i] not in wikipedia_dict:
            try:
                page = wikipedia.page(top_30_pages_test_dict[word][i])
                wikipedia_dict[top_30_pages_test_dict[word][i]] = page.content
            except:
                error_words.append(top_30_pages_test_dict[word][i])
                print('Error Word is: ', top_30_pages_test_dict[word][i])

1  ==>  Saracens
2  ==>  Buffalo Sabres
3  ==>  Newbridge
Error Word is:  newbridge (caerphilly) railway station
4  ==>  Montenegro
Error Word is:  house of petrović-njegoš
5  ==>  Czech
6  ==>  Ahmedou Ould Abdallah
7  ==>  Interfax
8  ==>  Brazilian
9  ==>  Zaragoza
10  ==>  Shannon
11  ==>  Patrick Vieira
12  ==>  Yugoslavia
Error Word is:  serbia national basketball team
13  ==>  PACIFIC


C:\Users\shubh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\shubh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error Word is:  pacific (album)
14  ==>  Aberdeen
Error Word is:  aberdeen (film)
15  ==>  Mafikeng
16  ==>  Gerard Van Velde
17  ==>  Michigan
18  ==>  ILO
19  ==>  WALLABY
20  ==>  Volendam
21  ==>  Middlesbrough
22  ==>  Wallaby
23  ==>  Roman
24  ==>  African
25  ==>  Caerphilly
26  ==>  Hawick
27  ==>  Okabe
28  ==>  Paul Gascoigne
29  ==>  Hosni Mubarak
30  ==>  Chirac
31  ==>  BUENOS AIRES
32  ==>  Peter van Vossen
33  ==>  Francesco Totti
34  ==>  British Columbia
35  ==>  Rowson
36  ==>  Maccabi Haifa
37  ==>  Brazilians
38  ==>  Syria
39  ==>  Davies
Error Word is:  steve davies (footballer)
40  ==>  Ales Valenta
41  ==>  Dean Jones
42  ==>  Juventus
43  ==>  Townsend
Error Word is:  union party (united states)
Error Word is:  james townsend
44  ==>  Enfield
Error Word is:  enfield (county meath) railway station
45  ==>  Minneapolis
Error Word is:  minneapolis_marines/minneapolis marines/red jackets
46  ==>  Sinton
47  ==>  Boroughmuir
48  ==>  Norwich
49  ==>  Suker
50  ==> 

274  ==>  Celtic
275  ==>  UEFA
276  ==>  Galley
Error Word is:  stu galley
277  ==>  LONDON
Error Word is:  london (1926 film)
278  ==>  Orlando
279  ==>  ORLANDO
280  ==>  Minnesota
Error Word is:  minnesota attorney general
281  ==>  McCoist
282  ==>  Zieleniec
283  ==>  Yugoslav
284  ==>  Chile
285  ==>  Japan
Error Word is:  japan national field hockey team
286  ==>  England
287  ==>  ADRs
288  ==>  MEXICO CITY
289  ==>  Zulu
Error Word is:  zulu
Error Word is:  zulu (film)
Error Word is:  zulu (song)
Error Word is:  george lloyd (aviator)
290  ==>  Hartlepool
Error Word is:  hartlepool by-election, 2004
291  ==>  N.IRELAND
292  ==>  ARSENAL
Error Word is:  arsenal (band)
293  ==>  Margaret Thatcher
294  ==>  Federal Reserve
295  ==>  Liberian
296  ==>  Primoz Peterka
297  ==>  Gloucester
298  ==>  Chris Sutton
299  ==>  Atlanta
300  ==>  Shimon Peres
301  ==>  Llanelli
302  ==>  Hwang Sun Hong
303  ==>  NEW YORK
Error Word is:  new york (typeface)
304  ==>  Tass
305  ==>  Rosario

505  ==>  Everton
506  ==>  Africa
Error Word is:  africa (album)
Error Word is:  africa (journal)
507  ==>  South Lebanon Army
508  ==>  African National Congress
509  ==>  HOBART
510  ==>  ALBRIGHT
Error Word is:  george albright
511  ==>  UNITED NATIONS
Error Word is:  united nations (album)
512  ==>  Nottingham Forest
513  ==>  Maccabi Petah Tikva
514  ==>  Chilean
515  ==>  Jacques Chirac
516  ==>  AUSTRALIA
517  ==>  Islam
Error Word is:  mohammad rafiqul islam
Error Word is:  rezaul islam
Error Word is:  rezwanul islam
Error Word is:  mafizul islam
518  ==>  Exeter
519  ==>  Premier league
520  ==>  Adnan Al Talyani
521  ==>  Kinshasa
522  ==>  SOUTH KOREA
Error Word is:  korea women's national basketball team
523  ==>  Wigan
524  ==>  Scunthorpe
525  ==>  OTTAWA
Error Word is:  ottawa (county of)
526  ==>  Salomon Brothers
527  ==>  Watford
528  ==>  Berisha
Error Word is:  valon berisha
Error Word is:  fadil berisha
529  ==>  HARARE
Error Word is:  1995 all-africa games
530  =

755  ==>  Roberto Mancini
756  ==>  Ivan Zamorano
757  ==>  OPEC
758  ==>  Sutton
Error Word is:  sutton railway station, ireland
759  ==>  Umkomaas
760  ==>  SAB
761  ==>  NEW DELHI
762  ==>  Organisation of the Islamic Conference
763  ==>  Elf
764  ==>  ENGLISH
765  ==>  U.S
766  ==>  Albion
767  ==>  Pavel Nedved
768  ==>  Conservative
Error Word is:  conservative christianity
Error Word is:  republican party (united states)
769  ==>  Boutros-Ghali
770  ==>  Michael DiVenuto
771  ==>  USDA
772  ==>  Kenneth Clarke
773  ==>  Udinese
774  ==>  Harper
Error Word is:  bill harper
Error Word is:  roy harper
775  ==>  Fiorentina
776  ==>  Movement for a Democratic Slovakia
777  ==>  Thoma
Error Word is:  vincenzo thoma
778  ==>  Racing Santander
779  ==>  Tallinn
780  ==>  Woolmer
781  ==>  West Bromwich
782  ==>  ICFTU
783  ==>  Manchester United
784  ==>  National League for Democracy
785  ==>  SINGAPORE
786  ==>  Clydebank
787  ==>  Melville
Error Word is:  melville (electoral district

Error Word is:  barnet (disambiguation)
960  ==>  NBA
961  ==>  Alloa
962  ==>  Belfast
Error Word is:  belfast (song)
963  ==>  Port Vale
964  ==>  DETROIT
965  ==>  Whelan
Error Word is:  noel whelan
Error Word is:  edward whelan
966  ==>  Wallabies
967  ==>  Erjon Bogdani
968  ==>  American Veterinary Medical Association
969  ==>  Bure
970  ==>  Veronica Brenner
971  ==>  Internet
Error Word is:  internet gaming gate
972  ==>  N. Ireland
973  ==>  Airdrieonians
974  ==>  World Trade Organisation
975  ==>  Lebanon
976  ==>  Russia
Error Word is:  handball union of russia
977  ==>  Dundee
978  ==>  Tasmanian
Error Word is:  tasmanian composers collective
979  ==>  Albanian Football Association
980  ==>  Rome
Error Word is:  1993 italian open (tennis)
Error Word is:  1992 italian open (tennis)
981  ==>  Axel Schulz
982  ==>  Bledar Kola
983  ==>  Jewish
984  ==>  Queens Park Rangers
985  ==>  Dean Sturridge
986  ==>  Clarence Seedorf
987  ==>  TALLINN
988  ==>  Estonia
Error Word is:  

In [9]:
with open('input/wikipedia_content_all.pkl', 'wb') as f:
    pickle.dump(wikipedia_dict, f, pickle.HIGHEST_PROTOCOL)


In [101]:
with open('input/wikipedia_content_all.pkl', 'rb') as f:
    wikipedia_dict = pickle.load(f)


In [ ]:
# Last error word in train: tibet (comics)
# First error word in test: newbridge (caerphilly) railway station

In [17]:
pd.DataFrame(error_words, columns=['PageName']).to_csv('Error_words.csv', index = False)
pd.DataFrame(error_words, columns=['PageName']).to_csv('input/Error_words.txt', index = False)

In [18]:
len(error_words)

2475

In [19]:
len(wikipedia_dict)

26558

In [ ]:
wikipedia_content = {}
for i in range(len(a['Brazilian'])):
    if a['Brazilian'][i] not in wikipedia_dict.keys:
        page = wikipedia.page(top_30_pages_train_dict[word][i])
        wikipedia_content[top_30_pages_train_dict[word][i]] = page.content

In [ ]:
top_30_pages_train_dict['The Times']

In [ ]:
data_dict.keys
data_dict[next(iter(data_dict))]


In [ ]:
imp_list_train_links = []
for word in imp_list_train[1:10]:
  try:
    imp_list_train_links.append(data_dict[word])
  except:
    pass

In [ ]:
imp_list_train_links[0]

In [ ]:
imp_list_train_content = []
for i in range(len(imp_list_train_links)):
  for j in range(len(imp_list_train_links[i])):
    page = wikipedia.page(imp_list_train_links[i][j])
    imp_list_train_content.append(page.content)

In [ ]:
try:
    page = wikipedia.page('The_Times_(South_Africa)')
    print(page.content)
except:
    print('Error')

In [ ]:
top

In [17]:
top_30_pages_train_dict

{'Tottenham': ['tottenham',
  'tottenham hotspur f.c.',
  'tottenham (uk parliament constituency)',
  'tottenham, new south wales',
  'tottenham, ontario',
  'tottenham, victoria',
  'municipal borough of tottenham',
  'tottenham railway station, melbourne',
  'tottenham hale station',
  'new tecumseth, ontario',
  'ss tottenham',
  'tottenham house',
  'broadwater farm riot',
  'london borough of haringey',
  'arthur loftus tottenham',
  'tottenham mandem',
  'tottenham, tottenham'],
 'Fraser': ['fraser river',
  'malcolm fraser',
  'division of fraser',
  'louis fraser',
  'neale fraser',
  'andy fraser',
  'clan fraser',
  'fraser, michigan',
  'fraser, colorado',
  'fraser, australian capital territory',
  'fraser, edmonton',
  'fraser, british columbia',
  'benton fraser',
  'richard fraser',
  'josh fraser',
  'nancy fraser',
  'peter fraser (new zealand politician)',
  'robin fraser',
  'sean fraser (jamaican footballer)',
  'charles fraser (rugby league)',
  'simon fraser (expl